In [1]:
import os
os.environ['PYTHONHASHSEED'] = '0'
import tensorflow as tf

import numpy as np
import random as rn

In [2]:
SEED = 123
def reset_random_seeds():
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    rn.seed(SEED)

reset_random_seeds()

In [3]:
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,BatchNormalization,Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from efficientnet.tfkeras import EfficientNetB7 #使用するモデルにあわせて変更する（B0~B7） 
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow_addons as tfa

from vit_keras import vit, utils

from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar10
import datetime

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
image_size = 32
input_shape=(image_size,image_size,3)
num_classes = 10 

In [6]:
def buildModel():
    model = Sequential()
    model.add(EfficientNetB7(include_top=False,
                             weights='imagenet',
                             input_shape=input_shape))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(num_classes, activation="softmax"))
    
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), 
                  loss="categorical_crossentropy", 
                  metrics=["accuracy"])
    
    return model

In [7]:
def buildModel_ViT():
    vit_model = vit.vit_b16( 
                image_size = image_size,
                activation = 'sigmoid',
                pretrained = True,
                include_top = False,
                pretrained_top = False)
    model = tf.keras.Sequential([
        vit_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(21, activation = tfa.activations.gelu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(num_classes, 'softmax')
        ],name = 'vision_transformer')
    
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), 
                  loss="categorical_crossentropy", 
                  metrics=["accuracy"])
    
    return model

In [8]:
def train_efficientnet(X, y, steps_per_epoch, epochs, batch_size, callbacks):

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True)
    y_train = to_categorical(y_train)
    y_valid = to_categorical(y_valid)

    datagen = ImageDataGenerator(rotation_range=20, 
                                 horizontal_flip=True, 
                                 width_shift_range=0.2, 
                                 zoom_range=0.2)
    
    train_generator = datagen.flow(X_train, y_train,batch_size=batch_size)

    model = buildModel()

    history = model.fit_generator(train_generator,
                        steps_per_epoch=steps_per_epoch,
                        epochs=epochs,
                        validation_data=(X_valid, y_valid),
                        callbacks=callbacks,
                        shuffle=True)
    
    return model, history

In [9]:
def train_vit(X, y, steps_per_epoch, epochs, batch_size, callbacks):
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True)
    y_train = to_categorical(y_train)
    y_valid = to_categorical(y_valid)
    
    datagen = ImageDataGenerator(rotation_range=20, 
                                 horizontal_flip=True, 
                                 width_shift_range=0.2, 
                                 zoom_range=0.2)
    
    train_generator = datagen.flow(X_train, y_train,batch_size=batch_size)
        
    model = buildModel_ViT()

    history = model.fit_generator(train_generator,
                        steps_per_epoch=steps_per_epoch,
                        epochs=epochs,
                        validation_data=(X_valid, y_valid),
                        callbacks=callbacks,
                        shuffle=True)
    
    return model, history

### Train EfficientNet

In [10]:
steps_per_epoch = 1250
epochs = 1000
batch_size = 32

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',factor=0.2,patience=2,verbose=1,
                              min_delta=1e-4,min_lr=1e-6,mode='max')
earlystopping = EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5,
                                                 mode='max', verbose=1)
callbacks = [earlystopping, reduce_lr]

print('Starting Time：',datetime.datetime.now())
model, history = train_efficientnet(x_train, y_train, steps_per_epoch, epochs, batch_size, callbacks)
print('Ending Time：',datetime.datetime.now())

Starting Time： 2021-05-16 15:30:57.181974
258441216/258434480 [==============================] - 16s 0us/step


/home/pongsakorn/tensorflow2/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/1000
1250/1250 [==============================] - 332s 245ms/step - loss: 2.0138 - accuracy: 0.2672 - val_loss: 1.1831 - val_accuracy: 0.5849
Epoch 2/1000
1250/1250 [==============================] - 303s 242ms/step - loss: 1.3127 - accuracy: 0.5409 - val_loss: 0.8599 - val_accuracy: 0.7121
Epoch 3/1000
1250/1250 [==============================] - 303s 243ms/step - loss: 1.0429 - accuracy: 0.6427 - val_loss: 0.6915 - val_accuracy: 0.7751
Epoch 4/1000
1250/1250 [==============================] - 305s 244ms/step - loss: 0.8940 - accuracy: 0.6946 - val_loss: 0.6030 - val_accuracy: 0.7988
Epoch 5/1000
1250/1250 [==============================] - 306s 245ms/step - loss: 0.7964 - accuracy: 0.7267 - val_loss: 0.5832 - val_accuracy: 0.8036
Epoch 6/1000
1250/1250 [==============================] - 307s 246ms/step - loss: 0.7012 - accuracy: 0.7646 - val_loss: 0.4973 - val_accuracy: 0.8320
Epoch 7/1000
1250/1250 [==============================] - 307s 245ms/step - loss: 0.6479 - accuracy:

In [12]:
preds = []
X = x_test
pred = model.predict(X)

df_pred = pd.DataFrame(pred)
pred = np.array(df_pred.idxmax(axis=1))
df_pred = pd.DataFrame(pred)
df_y = pd.DataFrame(y_test)
df_result = pd.concat([df_y, df_pred], axis=1)
df_result.columns = ['y','pred']
print(df_result)

      y  pred
0     3     3
1     8     8
2     8     8
3     0     8
4     6     6
...  ..   ...
9995  8     8
9996  3     3
9997  5     5
9998  1     1
9999  7     7

[10000 rows x 2 columns]


In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
print('Confusion Matrix:')
print(confusion_matrix(df_result['y'],df_result['pred']))
print('Accuracy :{:.4f}'.format(accuracy_score(df_result['y'],df_result['pred'])))
print('Precision:{:.4f}'.format(precision_score(df_result['y'],df_result['pred'],average='macro')))
print('Recall   :{:.4f}'.format(recall_score(df_result['y'],df_result['pred'],average='macro')))
print('F_score  :{:.4f}'.format(f1_score(df_result['y'],df_result['pred'],average='macro')))

Confusion Matrix:
[[903  15  12   3  10   0   4   6  30  17]
 [  4 954   1   1   0   0   3   1   5  31]
 [ 19   1 872  28  23   8  31  10   3   5]
 [  6   8  25 761  29  84  56  14   4  13]
 [  1   2  28  20 897   4  22  24   1   1]
 [  3   1  13 125  24 767  27  32   3   5]
 [  4   1  14  11   2   2 965   1   0   0]
 [  6   2   6  15  15  15   4 933   3   1]
 [ 27  14   4   4   0   0   1   0 933  17]
 [  6  38   0   2   0   1   1   2   5 945]]
Accuracy :0.8930
Precision:0.8926
Recall   :0.8930
F_score  :0.8921


### Train ViT

In [14]:
steps_per_epoch = 1250
epochs = 1000
batch_size = 32

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',factor=0.2,patience=2,verbose=1,
                              min_delta=1e-4,min_lr=1e-6,mode='max')
earlystopping = EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5,
                                                 mode='max', verbose=1)
callbacks = [earlystopping, reduce_lr]

print('Starting Time：',datetime.datetime.now())
model, history = train_vit(x_train, y_train, steps_per_epoch, epochs, batch_size, callbacks)
print('Ending Time：',datetime.datetime.now())

Starting Time： 2021-05-16 20:39:27.050198


/home/pongsakorn/tensorflow2/lib/python3.6/site-packages/vit_keras/utils.py:79: UserWarning: Resizing position embeddings from 24 to 2
  UserWarning,
/home/pongsakorn/tensorflow2/lib/python3.6/site-packages/tensorflow_addons/activations/gelu.py:80: UserWarning: Default value of `approximate` is changed from `True` to `False`
  "Default value of `approximate` is changed from `True` to `False`"
/home/pongsakorn/tensorflow2/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/1000
1250/1250 [==============================] - 197s 150ms/step - loss: 2.0654 - accuracy: 0.2775 - val_loss: 1.7395 - val_accuracy: 0.3805
Epoch 2/1000
1250/1250 [==============================] - 186s 149ms/step - loss: 1.7173 - accuracy: 0.3884 - val_loss: 1.6309 - val_accuracy: 0.4147
Epoch 3/1000
1250/1250 [==============================] - 190s 152ms/step - loss: 1.6309 - accuracy: 0.4206 - val_loss: 1.5713 - val_accuracy: 0.4333
Epoch 4/1000
1250/1250 [==============================] - 192s 154ms/step - loss: 1.5777 - accuracy: 0.4363 - val_loss: 1.4909 - val_accuracy: 0.4779
Epoch 5/1000
1250/1250 [==============================] - 193s 154ms/step - loss: 1.5148 - accuracy: 0.4622 - val_loss: 1.4491 - val_accuracy: 0.4878
Epoch 6/1000
1250/1250 [==============================] - 194s 155ms/step - loss: 1.4632 - accuracy: 0.4822 - val_loss: 1.4428 - val_accuracy: 0.4873
Epoch 7/1000
1250/1250 [==============================] - 195s 156ms/step - loss: 1.4491 - accuracy:

In [15]:
preds = []
X = x_test
pred = model.predict(X)

df_pred = pd.DataFrame(pred)
pred = np.array(df_pred.idxmax(axis=1))
df_pred = pd.DataFrame(pred)
df_y = pd.DataFrame(y_test)
df_result = pd.concat([df_y, df_pred], axis=1)
df_result.columns = ['y','pred']
print(df_result)

      y  pred
0     3     3
1     8     8
2     8     8
3     0     0
4     6     6
...  ..   ...
9995  8     8
9996  3     5
9997  5     3
9998  1     4
9999  7     7

[10000 rows x 2 columns]


In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
print('Confusion Matrix:')
print(confusion_matrix(df_result['y'],df_result['pred']))
print('Accuracy :{:.4f}'.format(accuracy_score(df_result['y'],df_result['pred'])))
print('Precision:{:.4f}'.format(precision_score(df_result['y'],df_result['pred'],average='macro')))
print('Recall   :{:.4f}'.format(recall_score(df_result['y'],df_result['pred'],average='macro')))
print('F_score  :{:.4f}'.format(f1_score(df_result['y'],df_result['pred'],average='macro')))

Confusion Matrix:
[[635  61  33  19  12  13  32  28 104  63]
 [ 31 709   7   6   5   5  14  25  23 175]
 [ 71  32 405  57  93  75 130  79  26  32]
 [ 25  41  64 327  43 218 147  62  20  53]
 [ 32  20  72  40 461  42 155 142  16  20]
 [ 22  12  53 180  48 444 102  98  12  29]
 [ 12  15  37  37  46  29 760  31  13  20]
 [ 21  24  17  37  46  61  37 694  10  53]
 [ 94  65   9  19  16   7   7   9 700  74]
 [ 37 188   4  14   4   8  14  33  35 663]]
Accuracy :0.5798
Precision:0.5778
Recall   :0.5798
F_score  :0.5722
